<a href="https://colab.research.google.com/github/hanyeob/fine_dust/blob/master/nsc_1_1_%EC%A0%84%EC%B2%98%EB%A6%AC%2B%EB%9D%BC%EB%B2%A8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

## 모듈 불러오기

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import plotly.express as px
import plotly.graph_objects as go

## 파일 불러오기

In [ ]:
isc_2_1='isc_2_1.csv'
isc_3_1='isc_3_1.csv'
isc_3_3='isc_3_3.csv'
nsc_1_1='nsc_1_1.csv'
nsc_1_2='nsc_1_2.csv'
nsc_2_5='nsc_2_5.csv'
sdc_1_1='sdc_1_1.csv'
sdc_2_1='sdc_2_1.csv'
sic_2_6='sic_2_6.csv'
sic_4_2='sic_4_2.csv'

choosefile = nsc_1_1
address = 'https://raw.githubusercontent.com/hanyeob/fine_dust/master/csv%20files/' + choosefile

In [ ]:
dataset = pd.read_csv(address)

In [4]:
dataset[0:5]

,tt,0.3_outdoor,0.5_outdoor,1.0_outdoor,3.0_outdoor,5.0_outdoor,10.0_outdoor,0.3_classroom,0.5_classroom,1.0_classroom,3.0_classroom,5.0_classroom,10.0_classroom,temp_outdoor,RH_outdoor,AH_outdoor,CO2_outdoor,temp_classroom,RH_classroom,AH_classroom,CO2_classroom,temp_meteo_425,RH_meteo_425,WD_meteo_425,WS_meteo_425,temp_meteo_889,RH_meteo_889,WD_meteo_889,WS_meteo_889,PM10_air_outdoor,PM2.5_air_outdoor,temp_air_outdoor,RH_air_outdoor,AH_air_outdoor,CO2_air_outdoor,VOCS_air_outdoor,Noise_air_outdoor,PM10_air_classroom,PM2.5_air_classroom,temp_air_classroom,RH_air_classroom,AH_air_classroom,CO2_air_classroom,VOCS_air_classroom,Noise_air_classroom
0,28-10-2019 18:53:36,160652300,17023680,1351590,105300,43463,1767,52001060,4746996,533216,49823,21201,3887,17.1,55.0,0.006682,441.377,NaN,NaN,NaN,NaN,14.7,72.9,182.3,1.7,14.6,67.9,219.3,0.5,15.0,8.0,18.3,51.0,0.006684,607.0,14.0,53,9.0,3.0,23.2,38.0,0.006736,515.0,0.0,52.0
1,28-10-2019 18:54:36,164037100,16869970,1345936,95760,45936,5300,54966430,4830036,547703,44523,17668,3180,17.1,56.0,0.006805,447.443,NaN,NaN,NaN,NaN,14.6,72.9,176.1,3.7,14.3,69.0,0.0,0.0,15.0,8.0,18.3,52.0,0.006816,613.0,37.0,53,9.0,3.0,23.2,38.0,0.006736,516.0,0.0,52.0
2,28-10-2019 18:55:36,160796800,16278450,1272792,100707,32862,3534,55765370,5035689,555124,54770,21555,2120,17.0,56.0,0.006762,443.399,NaN,NaN,NaN,NaN,14.6,73.2,183.9,2.5,14.1,69.9,0.0,0.0,13.0,7.0,18.2,52.0,0.006773,609.0,40.0,53,6.0,1.0,23.3,38.0,0.006777,514.0,0.0,53.0
3,28-10-2019 18:56:36,158039900,16036040,1285159,101413,46996,4947,50768200,4794700,527915,38516,26148,3180,17.0,56.0,0.006762,446.432,22.9,40.0,NaN,694.0,14.6,73.2,185.4,1.7,14.0,70.8,0.0,0.0,17.0,8.0,18.2,52.0,0.006773,612.0,36.0,53,5.0,1.0,23.4,38.0,0.006818,525.0,0.0,52.0
4,28-10-2019 18:57:36,157592200,16115900,1268905,93993,47703,4594,51080210,4900000,554417,50177,26148,3180,17.0,56.0,0.006762,449.465,22.9,40.0,NaN,600.0,14.7,73.3,181.0,2.4,13.9,71.4,0.0,0.0,24.0,9.0,18.3,52.0,0.006816,615.0,36.0,53,4.0,2.0,23.4,38.0,0.006818,521.0,1.0,53.0


## 데이터 정리

In [ ]:
dataset = dataset.loc[:,'tt':'CO2_classroom']
dataset = dataset.drop(columns=['temp_outdoor', 'RH_outdoor','AH_outdoor','temp_classroom', 'RH_classroom','AH_classroom'])
dataset

In [ ]:
dataset['tt'] = pd.to_datetime(dataset['tt'], format="%d-%m-%Y %H:%M:%S")
dataset.tt

0       2019-10-28 18:53:36
1       2019-10-28 18:54:36
2       2019-10-28 18:55:36
3       2019-10-28 18:56:36
4       2019-10-28 18:57:36
                ...        
11289   2019-11-05 15:24:30
11290   2019-11-05 15:25:30
11291   2019-11-05 15:26:30
11292   2019-11-05 15:27:30
11293   2019-11-05 15:28:30
Name: tt, Length: 11294, dtype: datetime64[ns]

## 불러온 데이터 NaN값 처리

## NaN값 확인

In [ ]:
dataset.isnull().sum()

### NaN값을 이후/이전에 측정된 값으로 채워넣기 

In [ ]:
for i in reversed(range(0,len(dataset))): 
    if(np.isnan(dataset.loc[i,'CO2_classroom'])):
      if(i+1 < len(dataset)):
        dataset.loc[i,'CO2_classroom'] = dataset.loc[i+1,'CO2_classroom']

for i in range(0,len(dataset)): 
    if(np.isnan(dataset.loc[i,'CO2_classroom'])):
      if(i > 0):
        dataset.loc[i,'CO2_classroom'] = dataset.loc[i-1,'CO2_classroom']

# 미세먼지, 초미세먼지 단위 맞춤 (Scaling)

In [ ]:
dataset['0.3_outdoor_scaled'] = dataset['0.3_outdoor'] * 0.3 ** 3
dataset['0.5_outdoor_scaled'] = dataset['0.5_outdoor'] * 0.5 ** 3
dataset['1.0_outdoor_scaled'] = dataset['1.0_outdoor'] * 1.0 ** 3
dataset['3.0_outdoor_scaled'] = dataset['3.0_outdoor'] * 3.0 ** 3
dataset['5.0_outdoor_scaled'] = dataset['5.0_outdoor'] * 5.0 ** 3
dataset['10.0_outdoor_scaled'] = dataset['10.0_outdoor'] * 10.0 ** 3

dataset['0.3_classroom_scaled'] = dataset['0.3_classroom'] * 0.3 ** 3
dataset['0.5_classroom_scaled'] = dataset['0.5_classroom'] * 0.5 ** 3
dataset['1.0_classroom_scaled'] = dataset['1.0_classroom'] * 1.0 ** 3
dataset['3.0_classroom_scaled'] = dataset['3.0_classroom'] * 3.0 ** 3
dataset['5.0_classroom_scaled'] = dataset['5.0_classroom'] * 5.0 ** 3
dataset['10.0_classroom_scaled'] = dataset['10.0_classroom'] * 10.0 ** 3

dataset

## 미세먼지/초미세먼지(내부,외부) 데이터 통합

In [ ]:
dataset['PM2.5_outdoor'] = np.sum(dataset.loc[:,'0.3_outdoor_scaled':'1.0_outdoor_scaled'], axis=1)
dataset['PM10_outdoor'] =  np.sum(dataset.loc[:,'3.0_outdoor_scaled':'10.0_outdoor_scaled'], axis=1)

dataset['PM2.5_classroom'] = np.sum( dataset.loc[:,'0.3_classroom_scaled':'1.0_classroom_scaled'], axis=1)
dataset['PM10_classroom'] = np.sum( dataset.loc[:,'3.0_classroom_scaled':'10.0_classroom_scaled'], axis=1)

dataset

In [ ]:
# CO2_fig = go.Figure()
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_outdoor'],mode='lines',name='CO2_outdoor'))
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_classroom'],mode='lines',name='CO2_classroom'))

In [ ]:
# PM_fig = go.Figure()
# from plotly.subplots import make_subplots
# PM_fig = make_subplots(specs=[[{"secondary_y": True}]])

# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM2.5_outdoor'],line=dict(color='red'), mode='lines',name='PM2.5_outdoors'))
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM10_outdoor'],line=dict(color='blue'),mode='lines',name='PM10_outdoors'),secondary_y=True,)
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM2.5_classroom'],line=dict(color='orange'),mode='lines',name='PM2.5_classroom'))
# PM_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['PM10_classroom'],line=dict(color='skyblue'),mode='lines',name='PM10_classroom'),secondary_y=True,)

# PM_fig.update_yaxes(title_text="PM 2.5", secondary_y=False)
# PM_fig.update_yaxes(title_text="PM10", secondary_y=True)

# 교실 내 변화량 column생성

In [ ]:
for i in range(1,len(dataset)) :
    dataset.loc[i,'PM2.5_classroom_change'] = dataset.loc[i,'PM2.5_classroom'] - dataset.loc[i-1,'PM2.5_classroom']
    dataset.loc[i,'PM10_classroom_change'] = dataset.loc[i,'PM10_classroom'] - dataset.loc[i-1,'PM10_classroom']
    dataset.loc[i,'CO2_classroom_change'] = dataset.loc[i,'CO2_classroom'] - dataset.loc[i-1,'CO2_classroom']

dataset.loc[0,'PM2.5_classroom_change'] =dataset.loc[1,'PM2.5_classroom_change']
dataset.loc[0,'PM10_classroom_change']  =dataset.loc[1,'PM10_classroom_change'] 
dataset.loc[0,'CO2_classroom_change']   =dataset.loc[1,'CO2_classroom_change']  

In [ ]:
dataset.loc[:,['CO2_classroom','CO2_classroom_change']]

In [ ]:
# CO2_fig = go.Figure()
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_outdoor'],mode='lines',name='CO2_outdoor'))
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_classroom'],mode='lines',name='CO2_classroom'))

# 라벨링


## 재실여부 라벨링

In [ ]:
dataset['attend_manual'] = np.NaN

In [ ]:
for i in range(1,len(dataset)) :
    if((dataset.loc[i,'CO2_classroom_change'] > 0) & (dataset.loc[i,'CO2_classroom'] > dataset.loc[i,'CO2_outdoor'])) :
        # 교실내 CO2가 외부 CO2보다 많으면서 교실내 CO2양 증가 
        dataset.loc[i,'attend_manual'] = 'Yes';
    elif((dataset.loc[i,'PM10_classroom_change'] > 0) & (dataset.loc[i,'PM10_classroom'] > dataset.loc[i,'PM10_outdoor'])):
        # 교실내 PM10이 외부 PM10보다 많으면서 교실내 PM10양 증가 
        dataset.loc[i,'attend_manual'] = 'Yes';
    

In [ ]:
# 하교시간
dataset.loc[((np.datetime64('2019-10-28T18:59') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-29T08:20'))),'attend_manual'] = 'No'
dataset.loc[((np.datetime64('2019-10-29T16:40') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-30T07:50'))),'attend_manual'] = 'No'
dataset.loc[((np.datetime64('2019-10-30T16:50') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-31T08:20'))),'attend_manual'] = 'No'
dataset.loc[((np.datetime64('2019-10-31T16:45') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-01T08:22'))),'attend_manual'] = 'No'
dataset.loc[((np.datetime64('2019-11-01T15:36') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-02T11:10'))),'attend_manual'] = 'No'
dataset.loc[((np.datetime64('2019-11-02T13:00') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-04T08:03'))),'attend_manual'] = 'No'
dataset.loc[((np.datetime64('2019-11-04T16:59') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-05T07:30'))),'attend_manual'] = 'No'

# 외부수업
dataset.loc[((np.datetime64('2019-10-31T09:45') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-31T10:27'))),'attend_manual'] = 'No'


In [ ]:
px.line(x=dataset.tt, y= dataset.attend_manual)

## 환기여부 라벨링

In [ ]:
dataset['vent_manual'] = np.NaN

In [ ]:
# CO2_fig = go.Figure()
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_outdoor'],mode='lines',name='CO2_outdoor'))
# CO2_fig.add_trace(go.Scatter(x=dataset['tt'], y=dataset['CO2_classroom'],mode='lines',name='CO2_classroom'))

In [ ]:
# 하교시간
dataset.loc[((np.datetime64('2019-10-28T18:59') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-29T08:20'))),'vent_manual'] = 'No'
dataset.loc[((np.datetime64('2019-10-29T16:40') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-30T07:50'))),'vent_manual'] = 'No'
dataset.loc[((np.datetime64('2019-10-30T16:50') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-31T08:20'))),'vent_manual'] = 'No'
dataset.loc[((np.datetime64('2019-10-31T16:45') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-01T08:22'))),'vent_manual'] = 'No'
dataset.loc[((np.datetime64('2019-11-01T15:36') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-02T11:10'))),'vent_manual'] = 'No'
dataset.loc[((np.datetime64('2019-11-02T13:00') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-04T08:03'))),'vent_manual'] = 'No'
dataset.loc[((np.datetime64('2019-11-04T16:59') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-05T07:30'))),'vent_manual'] = 'No'

# CO2 그래프 판단
dataset.loc[((np.datetime64('2019-10-29T09:46') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-29T09:49'))),'vent_manual'] = 'Yes'
dataset.loc[((np.datetime64('2019-10-29T10:11') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-29T10:14'))),'vent_manual'] = 'Yes'
dataset.loc[((np.datetime64('2019-10-29T10:35') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-29T10:53'))),'vent_manual'] = 'Yes'
dataset.loc[((np.datetime64('2019-10-29T12:11') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-29T12:39'))),'vent_manual'] = 'Yes'
dataset.loc[((np.datetime64('2019-10-29T12:51') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-29T13:04'))),'vent_manual'] = 'Yes'
dataset.loc[((np.datetime64('2019-10-29T14:10') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-29T14:21'))),'vent_manual'] = 'Yes'

dataset.loc[((np.datetime64('2019-10-30T09:39') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-30T09:47'))),'vent_manual'] = 'Yes'
dataset.loc[((np.datetime64('2019-10-30T10:16') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-30T11:09'))),'vent_manual'] = 'Yes'
dataset.loc[((np.datetime64('2019-10-30T11:48') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-30T12:22'))),'vent_manual'] = 'Yes'
dataset.loc[((np.datetime64('2019-10-30T12:44') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-30T15:00'))),'vent_manual'] = 'Yes'

dataset.loc[((np.datetime64('2019-10-31T08:53') < dataset['tt'])         
             &(dataset['tt'] < np.datetime64('2019-10-31T10:27'))),'vent_manual'] = 'Yes'
dataset.loc[((np.datetime64('2019-10-31T11:32') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-31T12:58'))),'vent_manual'] = 'Yes'
dataset.loc[((np.datetime64('2019-10-31T13:45') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-10-31T14:12'))),'vent_manual'] = 'Yes'

dataset.loc[((np.datetime64('2019-11-01T12:36') < dataset['tt'])              
             &(dataset['tt'] < np.datetime64('2019-11-01T12:50'))),'vent_manual'] = 'Yes'
dataset.loc[((np.datetime64('2019-11-01T13:45') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-01T14:28'))),'vent_manual'] = 'Yes'

dataset.loc[((np.datetime64('2019-11-02T11:55') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-02T12:14'))),'vent_manual'] = 'Yes'

dataset.loc[((np.datetime64('2019-11-04T09:41') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-04T10:26'))),'vent_manual'] = 'Yes'
dataset.loc[((np.datetime64('2019-11-04T11:22') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-04T11:48'))),'vent_manual'] = 'Yes'
dataset.loc[((np.datetime64('2019-11-04T12:08') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-04T16:28'))),'vent_manual'] = 'Yes'

dataset.loc[((np.datetime64('2019-11-05T09:42') < dataset['tt']) 
             &(dataset['tt'] < np.datetime64('2019-11-05T10:30'))),'vent_manual'] = 'Yes'



In [ ]:
sum(dataset['vent_manual'].isnull())

2141

In [ ]:
# px.line( y = dataset.vent_manual)

### NaN값 처리

In [ ]:
dataset['attend_manual'] = dataset['attend_manual'].fillna('Yes')
dataset['vent_manual'] = dataset['vent_manual'].fillna('No')

In [ ]:
dataset

# 스케일링 (일반화)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaled = scaler.fit_transform(dataset[
                ['CO2_outdoor','PM2.5_outdoor','PM10_outdoor',
               'CO2_classroom','PM2.5_classroom','PM10_classroom',
               'CO2_classroom_change','PM2.5_classroom_change','PM10_classroom_change']
              ])
scaled = pd.DataFrame(data=scaled,columns=['CO2_outdoor','PM2.5_outdoor','PM10_outdoor',
               'CO2_classroom','PM2.5_classroom','PM10_classroom',
               'CO2_classroom_change','PM2.5_classroom_change','PM10_classroom_change'])
scaled

,CO2_outdoor,PM2.5_outdoor,PM10_outdoor,CO2_classroom,PM2.5_classroom,PM10_classroom,CO2_classroom_change,PM2.5_classroom_change,PM10_classroom_change
0,-0.750901,-0.376166,-0.664752,-0.053840,-0.173534,-0.368886,0.000376,0.802441,-0.111542
1,-0.651311,-0.368743,-0.578181,-0.053840,-0.127902,-0.392020,0.000376,0.802441,-0.111542
2,-0.717705,-0.394918,-0.657075,-0.053840,-0.104114,-0.397347,0.000376,0.417551,-0.025802
3,-0.667910,-0.405225,-0.579820,-0.053840,-0.187722,-0.375940,0.000376,-1.474643,0.102923
4,-0.618115,-0.407273,-0.591050,-0.236745,-0.166810,-0.370302,-4.867928,0.366903,0.027000
...,...,...,...,...,...,...,...,...,...
11289,-0.319345,-0.460137,-0.660979,-0.213395,0.898747,-0.010709,0.000376,-0.132738,-0.327124
11290,-0.302747,-0.447882,-0.585765,-0.213395,0.822576,-0.060784,0.000376,-1.343611,-0.241262
11291,-0.020576,-0.431959,-0.640119,-0.213395,0.843127,0.000827,0.000376,0.360537,0.296507
11292,0.593561,-0.429493,-0.626311,-0.213395,0.804646,-0.037579,0.000376,-0.679548,-0.185075


## CSV저장

In [ ]:
scaled = pd.concat([dataset.tt, scaled, dataset[['attend_manual','vent_manual']]], axis=1)
scaled

,tt,CO2_outdoor,PM2.5_outdoor,PM10_outdoor,CO2_classroom,PM2.5_classroom,PM10_classroom,CO2_classroom_change,PM2.5_classroom_change,PM10_classroom_change,attend_manual,vent_manual
0,2019-10-28 18:53:36,-0.750901,-0.376166,-0.664752,-0.053840,-0.173534,-0.368886,0.000376,0.802441,-0.111542,Yes,No
1,2019-10-28 18:54:36,-0.651311,-0.368743,-0.578181,-0.053840,-0.127902,-0.392020,0.000376,0.802441,-0.111542,Yes,No
2,2019-10-28 18:55:36,-0.717705,-0.394918,-0.657075,-0.053840,-0.104114,-0.397347,0.000376,0.417551,-0.025802,Yes,No
3,2019-10-28 18:56:36,-0.667910,-0.405225,-0.579820,-0.053840,-0.187722,-0.375940,0.000376,-1.474643,0.102923,Yes,No
4,2019-10-28 18:57:36,-0.618115,-0.407273,-0.591050,-0.236745,-0.166810,-0.370302,-4.867928,0.366903,0.027000,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...
11289,2019-11-05 15:24:30,-0.319345,-0.460137,-0.660979,-0.213395,0.898747,-0.010709,0.000376,-0.132738,-0.327124,Yes,No
11290,2019-11-05 15:25:30,-0.302747,-0.447882,-0.585765,-0.213395,0.822576,-0.060784,0.000376,-1.343611,-0.241262,Yes,No
11291,2019-11-05 15:26:30,-0.020576,-0.431959,-0.640119,-0.213395,0.843127,0.000827,0.000376,0.360537,0.296507,Yes,No
11292,2019-11-05 15:27:30,0.593561,-0.429493,-0.626311,-0.213395,0.804646,-0.037579,0.000376,-0.679548,-0.185075,Yes,No


In [ ]:
scaled_data = pd.get_dummies(scaled); 
scaled_data.to_csv(nsc_1_1+'_labeled_scaled.csv')